In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm 
from scipy.optimize import minimize_scalar, minimize

## Load data

In [2]:
# Load data 

# Airpassengers
from statsforecast.utils import AirPassengers as ap 

# Electricity consumption 
import pandas as pd 
elec_df = pd.read_csv('tbats.files/PJMW_hourly.csv')
elec = elec_df['PJMW_MW']
elec = elec[0:(24*7*2+1)]
elec = np.array(elec)

# Calls 
calls_df = pd.read_csv('tbats.files/calls_data.csv')
calls = calls_df['calls']
calls = calls[-2001:]
calls = np.array(calls) 

/Users/marianamenchero/Documents/statsforecast/statsforecast/core.py:24: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Box Cox parameter 

In [3]:
def guer_cv(lam, x, season_length): 
   
   """Minimize this function to find the optimal parameter for the Box-Cox transformation"""
   
   period = np.round(np.max(season_length)) 
   n = len(x) 
   nyears = int(np.floor(n/period))
   nobs = np.floor(nyears*period)
   m = int(n-nobs)
   xmat = x[m:n].reshape((nyears, period))

   xmean = np.nanmean(xmat, axis=1)
   xsd = np.nanstd(xmat, axis=1)
   xrat = xsd/(xmean**(1-lam))
   
   return np.nanstd(xrat)/np.nanmean(xrat)


In [4]:
def guerrero(x, season_length, lower=-1, upper=2): 
    
    """Finds optimal paramater for Box-Cox transformation using Guerrero's method"""
    
    if np.any(x < 0): 
        raise ValueError("Guerrero's method for selecting a Box-Cox parameter (lambda) is for strictly positive data")
    
    max_freq = np.max(season_length)
    if len(x) <= 2*max_freq: 
        res = 1
    else: 
        lam = 0 # initial guess 
        opt = minimize_scalar(guer_cv, lam, args = (x, season_length), method = 'bounded', bounds = (lower, upper))
        res = opt.x

    return res

In [5]:
print(guerrero(ap, np.array([12])))
print(guerrero(elec, np.array([24, 24*7])))
print(guerrero(calls, np.array([169, 169*5])))

# Results in R: 
# -0.2947156
# -0.9999242
# -0.9999242


-0.2947229671935735
-0.9999965176267451
-0.9999965176267451


## Box Cox transformation

In [6]:
def BoxCox(x, BoxCox_lambda): 
    
    """Applies the Box-Cox transformation with parameter BoxCox_lambda"""
    
    if BoxCox_lambda == 0: 
        w = np.log(x)
    else: 
        w = np.sign(x)*((np.abs(x)**BoxCox_lambda)-1)
        w = w/BoxCox_lambda
        
    return w

In [7]:
ap_trans = BoxCox(ap, guerrero(ap, np.array([12])))
elec_trans = BoxCox(elec, guerrero(elec, np.array([24, 24*7])))
calls_trans = BoxCox(calls, guerrero(calls, np.array([169, 169*5])))

print(np.sum(ap_trans))
print(np.sum(elec_trans))
print(np.sum(calls_trans))

# Results in R
# 392.3874
# 336.9674
# 1987.612

392.3815136503149
336.9430776103611
1987.4733584199282


In [8]:
def InvBoxCox(w, BoxCox_lambda): 
    
    """Inverts Box-Cox transformation with parameter BoxCox_lambda"""
    
    if BoxCox_lambda == 0: 
        x = np.exp(w) 
    else: 
        sign = np.sign(BoxCox_lambda*w+1)
        x = np.abs(BoxCox_lambda*w+1)**(1/BoxCox_lambda)
        x = sign*x
        
    return x 

In [9]:
ap_inv = InvBoxCox(ap_trans, guerrero(ap, np.array([12])))
elec_inv = InvBoxCox(elec_trans, guerrero(elec, np.array([24, 24*7])))
calls_inv = InvBoxCox(calls_trans, guerrero(calls, np.array([169, 169*5])))

print(np.sum(ap_inv))
print(np.sum(elec_inv))
print(np.sum(calls_inv))

# Results in R
# 40363
# 1973874
# 375810

40363.0
1973874.0000000233
375810.00000000006


## Initial parameters                   

In [10]:
def starting_params(season_length, k_vector, use_trend, use_damped_trend, ar_coeffs, ma_coeffs): 
    
    alpha = 0.09

    if use_trend: 
        beta = 0.05 
        if use_damped_trend: 
            phi = 0.999 
        else: 
            phi = 1 
    else: 
        beta = None 
        phi = None 
        
    if season_length is not None: 
        gamma_one_v = np.zeros((len(season_length),))
        gamma_two_v = np.zeros((len(season_length),)) 
    else: 
        gamma_one_v = None 
        gamma_two_v = None 
        
    res = {
        'alpha': alpha,
        'gamma_one_v': gamma_one_v,
        'gamma_two_v': gamma_two_v,
        'beta': beta,
        'phi': phi
    }

    return res

In [11]:
def aux_params(season_length, k_vector, use_trend, ar_coeffs, ma_coeffs): 

    if use_trend: 
        adjBeta = 1 
        b = 0.0
    else: 
        adjBeta = 0 
        b = None 
        
    if season_length is not None: 
        s_vector = np.zeros((np.nansum(k_vector)*2,))
    else: 
        s_vector = None

    if ar_coeffs is not None: 
        d_vector = np.zeros((len(ar_coeffs),)) 
    else: 
        d_vector = None
        
    if ma_coeffs is not None: 
        epsilon_vector = np.zeros((len(ma_coeffs),)) 
    else: 
        epsilon_vector = None 

    res = {
        'adjBeta': adjBeta,
        'b': b,
        's_vector': s_vector,
        'd_vector': d_vector,
        'epsilon_vector': epsilon_vector,
    }

    return res

## Functions 

### findPQ

In [12]:
def findPQ(ar_coeffs, ma_coeffs): 
    
    """Find the length of the AR coefficients (p) and the length of the MA coefficients (q)"""
    
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
    else: 
        p = 0 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
    else: 
        q = 0 
    return p, q

### makeXMatrix

In [13]:
def makeXMatrix(b, s_vector, d_vector, epsilon_vector): 
    x = np.array([0.0])
    if b is not None: 
        x = np.append(x, b)
    if s_vector is not None: 
        x = np.concatenate((x, s_vector))
    if d_vector is not None: 
        x = np.concatenate((x, d_vector)) 
    if epsilon_vector is not None: 
        x = np.concatenate((x, epsilon_vector))
    
    return x 

### makeTBATSWMatrix

In [14]:
def makeTBATSWMatrix(phi, k_vector, ar_coeffs, ma_coeffs, tau): 
    p, q = findPQ(ar_coeffs, ma_coeffs) 

    adjPhi = 0 
    numSeasonal = 0 
    numCols = 1

    if phi is not None: 
        adjPhi = 1 
        numCols += 1
    if k_vector is not None: 
        numSeasonal += len(k_vector) 
        numCols += tau 
    if ar_coeffs is not None: 
        numCols += p
    if ma_coeffs is not None: 
        numCols += q

    w_transpose = np.zeros((1, numCols))

    if k_vector is not None: 
        position = adjPhi 
        for s in range(numSeasonal):
            for j in range(position+1, position+k_vector[s]+1): 
                w_transpose[0,j] = 1 
            position = position+(2*k_vector[s])

    w_transpose[0,0] = 1

    if adjPhi == 1: 
        w_transpose[0,1] = phi

    if ar_coeffs is not None: 
        for i in range(1,p+1): 
            w_transpose[0,adjPhi+tau+i] = ar_coeffs[i-1]

    if ma_coeffs is not None: 
        for i in range(1,q+1): 
            w_transpose[0,adjPhi+tau+p+i] = ma_coeffs[i-1] 

    w = np.transpose(w_transpose) 

    return w_transpose, w 

### updateTBATSGammaBold 

In [15]:
def updateTBATSGammaBold(gamma_bold, k_vector, gamma_one_v, gamma_two_v): 
    endPos = 0 
    numK = len(k_vector) 

    for i in range(numK):
        gamma_bold[0, endPos:endPos+k_vector[i]] = gamma_one_v[i]
        gamma_bold[0, endPos+k_vector[i]:endPos+2*k_vector[i]] = gamma_two_v[i]
        endPos += 2*k_vector[i]

    return gamma_bold

### updateTBATSGMatrix 

In [16]:
def updateTBATSGMatrix(g, gamma_bold, alpha, beta): 
    adjust_Beta = 0

    if alpha is not None:
        g[0, 0] = alpha

    if beta is not None:
        g[1, 0] = beta
        adjust_Beta = 1

    if gamma_bold is not None:
        g[adjust_Beta+1:adjust_Beta+gamma_bold.shape[1]+1,0] = np.transpose(gamma_bold).reshape((gamma_bold.shape[1],))
   
    return g 

### makeTBATSFMatrix

In [17]:
def makeAIMatrix(C, S, ks): 
    Ai = np.zeros((ks*2,ks*2))
    Ai[0:ks, 0:ks] = C 
    Ai[0:ks, ks:ks*2] = S 
    Ai[ks:ks*2, 0:ks] = -1*S
    Ai[ks:ks*2, ks:ks*2] = C 
    return Ai

In [18]:
def makeTBATSFMatrix(alpha, beta, phi, seasonal_periods, k_vector, gamma_bold, ar_coeffs, ma_coeffs): 

    p, q = findPQ(ar_coeffs, ma_coeffs) 

    # 1. Alpha row
    F = np.array([[1.0]])
    if beta is not None:
        F = np.concatenate((F, np.array([phi]).reshape(1,1)), axis = 1)
    if seasonal_periods is not None: 
        tau_v = 2*np.nansum(k_vector)
        zero_tau = np.zeros(tau_v,).reshape(1, tau_v)
        F = np.concatenate((F, zero_tau), axis = 1) 
    if ar_coeffs is not None: 
        alpha_phi = alpha*ar_coeffs
        alpha_phi = alpha_phi.reshape(F.shape[0], alpha_phi.shape[0]) 
        F = np.concatenate((F, alpha_phi), axis = 1)
    if ma_coeffs is not None: 
        alpha_theta = alpha*ma_coeffs
        alpha_theta = alpha_theta.reshape(F.shape[0], alpha_theta.shape[0]) 
        F = np.concatenate((F, alpha_theta), axis = 1)

    # 2. Beta row 
    if beta is not None: 
        beta_row = np.array([[0, phi]]) 
        if seasonal_periods is not None: 
            beta_row = np.concatenate((beta_row, zero_tau), axis = 1)
        if ar_coeffs is not None: 
            beta_phi = beta*ar_coeffs 
            beta_phi = beta_phi.reshape(beta_row.shape[0], beta_phi.shape[0])
            beta_row = np.concatenate((beta_row, beta_phi), axis = 1) 
        if ma_coeffs is not None: 
            beta_theta = beta*ma_coeffs 
            beta_theta = beta_theta.reshape(beta_row.shape[0], beta_theta.shape[0]) 
            beta_row = np.concatenate((beta_row, beta_theta), axis = 1)
        F = np.concatenate((F, beta_row), axis = 0)
    
    # 3. Seasonal row 
    if seasonal_periods is not None: 
        seasonal_row = np.transpose(zero_tau)
        if beta is not None: 
            seasonal_row = np.concatenate((seasonal_row, seasonal_row), axis = 1)

        # Make A matrix 
        A = np.zeros((tau_v, tau_v))
        last_pos = 0 
        for i in range(len(k_vector)): 
            ks = k_vector[i] 
            ms = seasonal_periods[i]
            if seasonal_periods[i] != 2:
                C = np.zeros((ks,ks))
                for j in range(1,ks+1): 
                    l = (2*np.pi*j)/ms 
                    C[(j-1),(j-1)] = np.cos(l)
            else: 
                C = np.zeros((1,1))
            S = np.zeros((ks,ks))
            for j in range(1,ks+1): 
                l = (2*np.pi*j)/ms 
                S[(j-1),(j-1)] = np.sin(l)
            Ai = makeAIMatrix(C, S, ks)         
            A[last_pos:(last_pos+2*k_vector[i]), last_pos:(last_pos+2*k_vector[i])] = Ai 
            last_pos = last_pos+(2*k_vector[i])

        seasonal_row = np.concatenate((seasonal_row, A), axis = 1)
    
        if ar_coeffs is not None: 
            B = np.outer(gamma_bold, ar_coeffs)
            seasonal_row = np.concatenate((seasonal_row, B), axis = 1)
        if ma_coeffs is not None: 
            C = np.outer(gamma_bold, ma_coeffs) 
            seasonal_row = np.concatenate((seasonal_row, C), axis = 1) 

        F = np.concatenate((F, seasonal_row), axis = 0)

    # 4. AR rows 
    if ar_coeffs is not None: 
        p = len(ar_coeffs) 
        ar_rows = np.zeros((p, 1)) 
        if beta is not None: 
            ar_rows = np.concatenate((ar_rows, ar_rows), axis = 1) 
        if seasonal_periods is not None: 
            ar_seasonal_zeros = np.zeros((p, tau_v))
            ar_rows = np.concatenate((ar_rows, ar_seasonal_zeros), axis = 1) 
        ident = np.eye(p-1) 
        ident = np.concatenate((ident, np.zeros((p-1, 1))), axis = 1) 
        ar_part = np.concatenate((ar_coeffs.reshape(1, ar_coeffs.shape[0]), ident), axis = 0)
        ar_rows = np.concatenate((ar_rows, ar_part), axis = 1)
        if ma_coeffs is not None: 
            q = len(ma_coeffs)
            ma_in_ar = np.zeros((p, q)) 
            ma_in_ar[0,:] = ma_coeffs 
            ar_rows = np.concatenate((ar_rows, ma_in_ar), axis = 1)
        F = np.concatenate((F, ar_rows), axis = 0)

    # 5. MA rows 
    if ma_coeffs is not None: 
        q = len(ma_coeffs) 
        ma_rows = np.zeros((q, 1)) 
        if beta is not None: 
            ma_rows = np.concatenate((ma_rows, ma_rows), axis = 1)
        if seasonal_periods is not None: 
            ma_seasonal = np.zeros((q, tau_v))
            ma_rows = np.concatenate((ma_rows, ma_seasonal), axis = 1) 
        if ar_coeffs is not None: 
            ar_in_ma = np.zeros((q, p)) 
            ma_rows = np.concatenate((ma_rows, ar_in_ma), axis = 1) 
        ident = np.eye(q-1)
        ident = np.concatenate((ident, np.zeros((q-1, 1))), axis = 1) 
        ma_part = np.concatenate((np.zeros((1,q)), ident), axis = 0) 
        ma_rows = np.concatenate((ma_rows, ma_part), axis = 1) 
        F = np.concatenate((F, ma_rows), axis = 0)

    return F 
    

### calcTBATSFaster

In [19]:
def calcTBATSFaster(y_trans, w_transpose, g, F, x_nought): 

    n = y_trans.shape[0] 
    dimF = F.shape[0] 
    
    yhat = np.zeros((1,n)) # fitted values 
    e = np.zeros((1,n)) # residuals 
    x = np.zeros((len(x_nought),n)) # unoserved state vectors x_t 

    yhat[0,0] = np.dot(w_transpose, x_nought).reshape(())
    e[0,0] = y_trans[0]-yhat[0,0] 
    x[:,0] = np.dot(F, x_nought)+(g*e[0,0]).reshape((dimF,))

    for t in range(1,n): 
        yhat[:, t] = np.dot(w_transpose, x[:, (t-1)])
        e[0,t] = y_trans[t]-yhat[0,t]
        x[:,t] = np.dot(F,x[:,(t-1)].reshape((dimF,1))+g*e[0,t]).reshape((dimF,))
        
    return yhat, e, x

### extract_params 

In [20]:
def extract_params(params_vec, use_boxcox, use_trend, use_damped_trend, use_arma_errors, season_length, p, q): 

    if use_boxcox: 
        BoxCox_lambda = params_vec[0]
        alpha = params_vec[1]
        index = 2 
    else: 
        BoxCox_lambda = None
        alpha = params_vec[0]
        index = 1

    gamma_one_v = params_vec[index:index+len(season_length)]
    gamma_two_v = params_vec[index+len(season_length):index+2*len(season_length)]

    if use_trend and use_damped_trend and use_arma_errors: 
        beta = params_vec[index+2*len(season_length)]
        phi = params_vec[index+2*len(season_length)+1]
        ar_coeffs = params_vec[index+2*len(season_length)+2:index+2*len(season_length)+2+p]
        ma_coeffs = params_vec[index+2*len(season_length)+2+p:index+2*len(season_length)+2+p+q]

    if use_trend and use_damped_trend and (not use_arma_errors): 
        beta = params_vec[index+2*len(season_length)]
        phi = params_vec[index+2*len(season_length)+1]
        ar_coeffs = None 
        ma_coeffs = None 

    if use_trend and (not use_damped_trend) and use_arma_errors:
        beta = params_vec[index+2*len(season_length)]
        phi = None
        ar_coeffs = params_vec[index+2*len(season_length)+1:index+2*len(season_length)+1+p]
        ma_coeffs = params_vec[index+2*len(season_length)+1+p:index+2*len(season_length)+1+p+q]

    if use_trend and (not use_damped_trend) and (not use_arma_errors):
        beta = params_vec[index+2*len(season_length)]
        phi = None
        ar_coeffs = None
        ma_coeffs = None

    if (not use_trend) and (not use_damped_trend) and use_arma_errors:
        beta = None
        phi = None
        ar_coeffs = params_vec[index+2*len(season_length):index+2*len(season_length)+p]
        ma_coeffs = params_vec[index+2*len(season_length)+p:index+2*len(season_length)+p+q]

    if (not use_trend) and (not use_damped_trend) and (not use_arma_errors):
        beta = None
        phi = None
        ar_coeffs = None
        ma_coeffs = None

    return BoxCox_lambda, alpha, gamma_one_v, gamma_two_v, beta, phi, ar_coeffs, ma_coeffs

### updateWtransposeMatrix

In [21]:
def updateWtransposeMatrix(w_transpose, phi, tau, ar_coeffs, ma_coeffs):
    p, q = findPQ(ar_coeffs, ma_coeffs) 
    adjBeta = 0 
    if phi is not None: 
        adjBeta = 1 
        w_transpose[0,1] = phi 
    if p > 0: 
        for i in range(1,p+1): 
            w_transpose[0, adjBeta+tau+i] = ar_coeffs[i-1]
        if q > 0: 
            for i in range(1,q+1): 
                w_transpose[0, adjBeta+tau+p+i] = ma_coeffs[i-1]
    elif q > 0: 
        for i in range(1,q+1): 
            w_transpose[0, adjBeta+tau+i] = ma_coeffs[i-1]
    
    return w_transpose

### updateTBATSGammaBold

In [22]:
def updateTBATSGammaBold(gamma_bold, k_vector, gamma_one_v, gamma_two_v): 
    endPos = 0 

    for i in range(len(k_vector)): 
        for j in range(endPos, k_vector[i]+endPos):
            gamma_bold[0,j] = gamma_one_v[i] 
        for j in range(k_vector[i]+endPos, 2*k_vector[i]+endPos): 
            gamma_bold[0,j] = gamma_two_v[i]
        endPos += 2*k_vector[i] 

    return gamma_bold 

### updateTBATSGMatrix 

In [23]:
def updateTBATSGMatrix(g, gamma_bold, alpha, beta): 
    adjust_beta = 0 
    g[0,0] = alpha 
    if beta is not None: 
        g[1,0] = beta 
        adjust_beta = 1 
    if gamma_bold is not None:  
        g[adjust_beta+1:adjust_beta+1+gamma_bold.shape[1], 0] = np.transpose(gamma_bold).reshape((gamma_bold.shape[1],))
    return g 

### updateFMatrix 

In [24]:
def updateFMatrix(F, phi, alpha, beta, gamma_bold, ar_coeffs, ma_coeffs, tau):
    if beta is not None: 
        F[0,1] = phi 
        F[1,1] = phi 
        betaAdjust = 1
    else: 
        betaAdjust = 0 
    p, q = findPQ(ar_coeffs, ma_coeffs)
    if ar_coeffs is not None:
        F[0, betaAdjust+tau+1:betaAdjust+tau+1+p] = alpha*ar_coeffs
        if betaAdjust == 1:  
            F[1, betaAdjust+tau+1:betaAdjust+tau+1+p] = beta*ar_coeffs 
        if tau > 0: 
            B = np.dot(gamma_bold.reshape((gamma_bold.shape[1],1)), ar_coeffs.reshape((1,ar_coeffs.shape[0])))
            F[1+betaAdjust:betaAdjust+tau+1, betaAdjust+tau+1:betaAdjust+tau+p+1] = B 
        F[betaAdjust+tau+1, betaAdjust+tau+1:betaAdjust+tau+p+1] = ar_coeffs 
    if ma_coeffs is not None: 
        F[0, betaAdjust+tau+p+1:betaAdjust+tau+p+q+1] = alpha*ma_coeffs 
        if betaAdjust == 1:  
            F[1, betaAdjust+tau+p+1:betaAdjust+tau+p+q+1] = beta*ma_coeffs
        if tau > 0: 
            C = np.dot(gamma_bold.reshape((gamma_bold.shape[1],1)),ma_coeffs.reshape((1,ma_coeffs.shape[0]))) 
            F[1+betaAdjust:betaAdjust+tau+1, betaAdjust+tau+p+1:betaAdjust+tau+p+q+1] = C 
        if ar_coeffs is not None: 
            F[betaAdjust+tau+1,(betaAdjust+tau+p+1):(betaAdjust+tau+p+q+1)] = ma_coeffs 

    return F 

### checkAdmissibility

In [25]:
def checkAdmissibility(BoxCox_lambda, bc_lower_bound, bc_upper_bound, phi, ar_coeffs, ma_coeffs, D): 
    if BoxCox_lambda is not None: 
        if (BoxCox_lambda <= bc_lower_bound) or (BoxCox_lambda >= bc_upper_bound): 
            return False 

    if phi is not None: 
        if (phi < 0.8) or (phi > 1): 
            return False 
        
    if ar_coeffs is not None:
        arlags = np.where(np.abs(ar_coeffs) > 1e-08)[0]
        if len(arlags) > 0:
            p = max(arlags)
            if np.min(np.abs(np.roots([1] + [-i for i in ar_coeffs[:p]]))) < 1 + 1e-2:
                return False

    if ma_coeffs is not None:
        malags = np.where(np.abs(ma_coeffs) > 1e-08)[0]
        if len(malags) > 0:
            q = max(malags)
            if np.min(np.abs(np.roots([1] + ma_coeffs[:q]))) < 1 + 1e-2:
                return False
  
    D_eigen_values = np.linalg.eigvals(D)
    return np.all(np.abs(D_eigen_values) < 1+1e-2)

### calcLikelihoodTBATS

In [26]:
def calcLikelihoodTBATS(params_vec, use_boxcox, lower, upper, use_trend, use_damped_trend, use_arma_errors, y_trans, season_length, k_vector, tau, p, q, w_transpose, gamma_bold, g, F, x_nought): 
    
    # Extract parameters 
    BoxCox_lambda, alpha, gamma_one_v, gamma_two_v, beta, phi, ar_coeffs, ma_coeffs = extract_params(params_vec, use_boxcox, use_trend, use_damped_trend, use_arma_errors, season_length, p, q)

    w_transpose = updateWtransposeMatrix(w_transpose, phi, tau, ar_coeffs, ma_coeffs) 
    gamma_bold = updateTBATSGammaBold(gamma_bold, k_vector, gamma_one_v, gamma_two_v)
    g = updateTBATSGMatrix(g, gamma_bold, alpha, beta)
    F = updateFMatrix(F, phi, alpha, beta, gamma_bold, ar_coeffs, ma_coeffs, tau)
    
    _, e, x = calcTBATSFaster(y_trans, w_transpose, g, F, x_nought)
    
    if use_boxcox: 
        log_likelihood = len(y_trans)*np.log(np.nansum(e**2))-2*(BoxCox_lambda-1)*np.nansum(np.log(y_trans)) 
    else: 
        log_likelihood = len(y_trans)*np.log(np.nansum(e**2))

    D = F-np.dot(g, w_transpose)

    if checkAdmissibility(BoxCox_lambda, lower, upper, phi, ar_coeffs, ma_coeffs, D): 
        return log_likelihood 
    else: 
        return 10**20

## Main 

In [55]:
y = ap
season_length = np.array([12])

use_boxcox = True
lower = 0
upper = 1.5
use_trend = True
use_damped_trend = True
use_arma_errors = False 

In [56]:
# TBATS model 

season_length = np.sort(season_length)
k_vector = np.full((len(season_length),), fill_value = 1) 

# Check if there are missing values 
indices = np.where(np.isnan(y))[0]
if len(indices) > 0: 
    max_index = indices[-1]
    y = y[max_index+1:len(y)]

# Check if there are negative values 
if np.any(y < 0): 
    use_boxcox = False 

# Box-Cox transformation 
if use_boxcox:
    BoxCox_lambda = guerrero(y, season_length, lower, upper) 
    y_trans = BoxCox(y, BoxCox_lambda) 
else: 
    BoxCox_lambda = None
    y_trans = y.copy()

#  ---------- First model (No ARMA errors) ----------
ar_coeffs = None 
ma_coeffs = None 

params = starting_params(season_length, k_vector, use_trend, use_damped_trend, ar_coeffs, ma_coeffs)
params['BoxCox_lambda'] = BoxCox_lambda
params['ar_coeffs'] = ar_coeffs
params['ma_coeffs'] = ma_coeffs

vals = aux_params(season_length, k_vector, use_trend, ar_coeffs, ma_coeffs)

# model_generator(y_trans, use_boxcox, use_trend, use_damped_trend, season_length, k_vector, params, vals)


In [57]:
params 

{'alpha': 0.09,
 'gamma_one_v': array([0.]),
 'gamma_two_v': array([0.]),
 'beta': 0.05,
 'phi': 0.999,
 'BoxCox_lambda': 5.526022037850897e-06,
 'ar_coeffs': None,
 'ma_coeffs': None}

### Model generator 

In [58]:
# Model generator
if season_length is not None: 
    tau = 2*sum(k_vector) 
    tau = tau.astype('int')
else: 
    tau = 0

x_nought = makeXMatrix(vals['b'], vals['s_vector'], vals['d_vector'], vals['epsilon_vector'])
num_seed_states = len(x_nought)

# w 
w_transpose, _ = makeTBATSWMatrix(params['phi'], k_vector, params['ar_coeffs'], params['ma_coeffs'], tau)

# gamma_bold 
if season_length is not None: 
    gamma_bold = np.zeros((1, 2*np.nansum(k_vector)))
    gamma_bold = updateTBATSGammaBold(gamma_bold, k_vector, params['gamma_one_v'], params['gamma_two_v'])
else: 
    gamma_bold = None  

p, q = findPQ(params['ar_coeffs'], params['ma_coeffs'])

# g 
g = np.zeros((2*np.nansum(k_vector)+1+vals['adjBeta']+p+q, 1)) 
if p != 0: 
    g[(vals['adjBeta']+tau+1),0] = 1 
if q != 0: 
    g[(vals['adjBeta']+tau+p+1),0] = 1
g = updateTBATSGMatrix(g, gamma_bold, params['alpha'], params['beta'])

# F 
F = makeTBATSFMatrix(params['alpha'], params['beta'], params['phi'], season_length, k_vector, gamma_bold, params['ar_coeffs'], params['ma_coeffs']) 

# D 
D = F-np.dot(g, w_transpose)


In [59]:
print(D)
print(D.shape)
print(np.sum(D))
print(np.sum(D, axis = 0))
print(np.sum(D, axis = 1))

[[ 0.91       0.90909   -0.09       0.       ]
 [-0.05       0.94905   -0.05       0.       ]
 [ 0.         0.         0.8660254  0.5      ]
 [ 0.         0.        -0.5        0.8660254]]
(4, 4)
4.310190807568877
[0.86      1.85814   0.2260254 1.3660254]
[1.72909   0.84905   1.3660254 0.3660254]


In [60]:
yhat, e, x = calcTBATSFaster(y_trans, w_transpose, g, F, x_nought)

In [61]:
print(np.sum(yhat))
print(np.sum(e))
print(np.sum(x))

797.996030157249
0.08960597071022125
804.2255105896488


In [62]:
w_tilda_transpose = np.zeros((len(y_trans), w_transpose.shape[1]))
w_tilda_transpose[0,:] = w_transpose 

for k in range(1, w_tilda_transpose.shape[0]): 
    w_tilda_transpose[k,:] = np.dot(w_tilda_transpose[k-1,:], D)

if p != 0 or q!= 0: 
    end_cut = w_tilda_transpose.shape[1] 
    start_cut = end_cut-(p+q)
    cols = np.arange(0, w_tilda_transpose.shape[1], 1)
    new_cols = np.delete(cols, np.arange(start_cut, end_cut, 1))
    w_tilda_transpose = w_tilda_transpose[:, new_cols]

model = sm.OLS(e.reshape((e.shape[1],1)), w_tilda_transpose).fit()
x_nought = model.params

if (p != 0 or q != 0):
    arma_seed_states = np.zeros((p+q,))
    x_nought = np.concatenate((x_nought, arma_seed_states))

In [63]:
model.params 

array([ 3.97266942, -0.01848245, -0.12434706,  0.06784968])

In [64]:
# This generates the same result as above 
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression(fit_intercept=False)
coefs = linear_regression.fit(w_tilda_transpose, e.reshape((e.shape[1],1))).coef_
coefs 

array([[ 3.97266942, -0.01848245, -0.12434706,  0.06784968]])

In [65]:
x_nought 

array([ 3.97266942, -0.01848245, -0.12434706,  0.06784968])

In [66]:
# Optimization 
if use_boxcox: 
    params_vec = np.array([params['BoxCox_lambda'], params['alpha']])
else: 
    params_vec = np.array([params['alpha']])
params_vec = np.hstack((params_vec, params['gamma_one_v'], params['gamma_two_v']))
if use_trend: 
    params_vec = np.hstack((params_vec, params['beta']))
if use_damped_trend: 
    params_vec = np.hstack((params_vec, params['phi']))
if ar_coeffs is not None: 
    params_vec = np.hstack((params_vec, params['ar_coeffs']))
if ma_coeffs is not None:
    params_vec = np.hstack((params_vec, params['ma_coeffs']))


In [67]:
params_vec 

array([5.52602204e-06, 9.00000000e-02, 0.00000000e+00, 0.00000000e+00,
       5.00000000e-02, 9.99000000e-01])

### Optimization

In [68]:
# optim_params = minimize(calcLikelihoodTBATS,
#                         params_vec,
#                         args = (use_boxcox, lower, upper, use_trend, use_damped_trend, use_arma_errors, y_trans, season_length, k_vector, tau, p, q, w_transpose, gamma_bold, g, F, x_nought), 
#                         method = 'Nelder-Mead',
#                         options = {'maxiter': 2000, 'disp': False}
#                         ).x

In [69]:
optim_params = minimize(calcLikelihoodTBATS,
                        params_vec,
                        args = (use_boxcox, lower, upper, use_trend, use_damped_trend, use_arma_errors, y_trans, season_length, k_vector, tau, p, q, w_transpose, gamma_bold, g, F, x_nought)
                        ).x

In [70]:
optim_BoxCox_lambda, optim_alpha, optim_gamma_one_v, optim_gamma_two_v, optim_beta, optim_phi, optim_ar_coeffs, optim_ma_coeffs = extract_params(optim_params, use_boxcox, use_trend, use_damped_trend, use_arma_errors, season_length, p, q)


In [71]:
print('Optimal Box-Cox lambda: ', optim_BoxCox_lambda)
print('Optimal alpha: ', optim_alpha)
print('Optimal gamma one vector: ', optim_gamma_one_v)
print('Optimal gamma two vector: ', optim_gamma_two_v)
print('Optimal beta: ', optim_beta)
print('Optimal phi: ', optim_phi)
print('Optimal AR coefficients: ', optim_ar_coeffs)
print('Optimal MA coefficients: ', optim_ma_coeffs)

Optimal Box-Cox lambda:  0.1620769204516143
Optimal alpha:  0.1576430385045225
Optimal gamma one vector:  [-0.01624938]
Optimal gamma two vector:  [-0.00868101]
Optimal beta:  0.05130967079710276
Optimal phi:  0.9654190388206612
Optimal AR coefficients:  None
Optimal MA coefficients:  None


In [72]:
new_w_transpose, _ = makeTBATSWMatrix(optim_phi, k_vector, optim_ar_coeffs, optim_ma_coeffs, tau)
new_gamma_bold = updateTBATSGammaBold(gamma_bold, k_vector, optim_gamma_one_v, optim_gamma_two_v)
new_g = updateTBATSGMatrix(g, new_gamma_bold, optim_alpha, optim_beta)
new_F = updateFMatrix(F, optim_phi, optim_alpha, optim_beta, new_gamma_bold, optim_ar_coeffs, optim_ma_coeffs, tau)

fitted, errors, x = calcTBATSFaster(y_trans, new_w_transpose, new_g, new_F, x_nought)

In [73]:
# Calculate log-likelihood 
if use_boxcox: 
    log_likelihood = len(y_trans)*np.log(np.nansum(errors**2))-2*(BoxCox_lambda-1)*np.nansum(np.log(y)) 
else: 
    log_likelihood = len(y_trans)*np.log(np.nansum(errors**2))

In [74]:
log_likelihood

1763.5644545855328

## TBATS Python library

In [68]:
from tbats import TBATS

estimator = TBATS(seasonal_periods = [12])

In [69]:
fitted_model = estimator.fit(ap)

In [70]:
print(fitted_model.summary())

Use Box-Cox: True
Use trend: True
Use damped trend: False
Seasonal periods: [12.]
Seasonal harmonics [5]
ARMA errors (p, q): (0, 0)
Box-Cox Lambda 0.000000
Smoothing (Alpha): 0.762786
Trend (Beta): 0.035132
Damping Parameter (Phi): 1.000000
Seasonal Parameters (Gamma): [-2.67397734e-07  6.91466869e-08]
AR coefficients []
MA coefficients []
Seed vector [ 4.81003417 -0.00858031 -0.14832239  0.05633621 -0.00904648  0.01081748
  0.00562992  0.02727922  0.05868876 -0.02752747 -0.03212159 -0.02132667]

AIC 1399.553947


In [80]:
# Example optimization 
def f(x):
    return x[0]**2 + x[1]**2

x0 = np.array([1.0, 2.0, 3.0, 4.0])
result = minimize(f, x0, method='BFGS')
print(result.x)

[-1.06541975e-08 -2.13083950e-08  3.00000000e+00  4.00000000e+00]
